# Testando BD Vetorial

## Calculando embeddings

Aqui usaremos um LLM para calcular os embeddings antes de inserir no BD.

In [16]:
import pandas as pd
import chromadb
import numpy as np
import tqdm

import os
from dotenv import load_dotenv
load_dotenv()

True

## Get embedding function

Vou usar o Gemini porque é gratuito até certo limite. Pegue sua chave de API no [Google AI Studio](https://aistudio.google.com/) e coloque no arquivo `.env`.

In [17]:
import chromadb.utils.embedding_functions as embedding_functions

google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.environ["GEMINI_API_KEY"])

In [18]:
# Load Chroma DB with the new embedding function.
client = chromadb.PersistentClient("./chroma.db")
collection = client.get_or_create_collection(name="01-embeddings", embedding_function=google_ef)

## Save data

In [19]:
df = pd.read_csv("./data/comments.csv")
print(f"df.columns = {list(df.columns)}")

df.columns = ['video_id', 'user', 'date', 'comment', 'likes', 'cleaned_comment', 'cluster']


In [22]:
documents = []
metadatas = []
for index, row in df.iterrows():
    documents.append(row['comment'])
    metadatas.append({"date": row['date'], "user": row['user'], "video_id": row['video_id']})

len(documents), len(metadatas) 

(2497, 2497)

In [24]:
count = collection.count()
print(f"Collection already contains {count} documents")

ids = [str(i) for i in range(count, count + len(documents))]

# Load the documents in batches of 100
for i in tqdm.tqdm(
    range(0, len(documents), 100), desc="Adding documents", unit_scale=100
):
    collection.add(
        ids=ids[i : i + 100],
        documents=documents[i : i + 100],
        metadatas=metadatas[i : i + 100],  # type: ignore
    )

new_count = collection.count()
print(f"Added {new_count - count} documents")

Collection already contains 0 documents


Adding documents:   0%|          | 0/2500 [00:00<?, ?it/s]

Adding documents: 100%|██████████| 2500/2500 [12:44<00:00,  3.27it/s]

Added 2497 documents


## Search

In [27]:
queries = ["melhor do youtube"]
n_res = 20

results = collection.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=n_res # how many results to return
)
results["documents"]

[['melhores vídeos',
  'Boa  explicação, para mim a melhor do youtube!',
  'Tem que continuar os vídeos, são os melhores',
  'bom vídeo',
  'Perfeito, ótimo vídeo',
  'Muito bom seus vídeos! Parabéns!',
  'Ótimo vídeo',
  'irmao vc e o melhor',
  'muito bom, este e o vídeo anterior',
  'Muito bom os vídeos, parabéns.',
  'Excelente vídeo 👏👏👏',
  'não terá mais videos ? òtimo canal.',
  'Poste mais videos disso por favor <3',
  'Terá mais videos?',
  'poe o arquivo desse video',
  'Ótimo vídeo!',
  'Seus vídeos Estão Min Ajudando Muito 👊',
  'uma das melhores explicações de programação do you tube!',
  'Excelente video',
  'Parabéns vídeo muito bom, obrigado!']]